# Intrusion Detection using CNN1N for CICIDS 2017 Data Set with 78 Features 

We use the pre-processing dataset from mlp4nids (Multi-layer perceptron for network intrusion detection) https://github.com/ArnaudRosay/mlp4nids. The train, test and validation sets have connection-based new features. The data splitting rate is 50:25:25

In [ ]:
# data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
# scaler = MinMaxScaler()
# print(scaler.fit(data))
# MinMaxScaler()
# print(scaler.data_max_)
# print(scaler.data_min_)
# print(scaler.transform(data))
# print(scaler.fit_transform([[2, 2]]))

In [ ]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 33kB/s 
     |████████████████████████████████| 2.9MB 45.6MB/s 
     |████████████████████████████████| 460kB 43.1MB/s 
     |████████████████████████████████| 3.0MB 38.1MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [ ]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [ ]:
keras.__version__

'2.3.0-tf'

In [ ]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [ ]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [ ]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [ ]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

## Step 1. Loading csv files

In [ ]:
# All columns
col_names = np.array(['dst sport count', 'src dport count', 'dst src count', 'dport count', 'sport count', 'dst host count','src host count','Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

### Option 1. Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load three csv files generated by mlp4nids (Multi-layer perceptron for network intrusion detection )
# first load the train set
df_train = pd.read_csv('/content/drive/My Drive/CICIDS2017/train_set_ext78_2.csv',names=col_names, skiprows=1)  

In [ ]:
print('Train set size: ', df_train.shape)

Train set size:  (879589, 79)


In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/CICIDS2017/test_set_ext78_2.csv',names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)

df_val = pd.read_csv('/content/drive/My Drive/CICIDS2017/crossval_set_ext78_2.csv',names=col_names, skiprows=1)  
print('Validation set size: ', df_val.shape)

Test set size:  (188483, 79)
Validation set size:  (188484, 79)


### Option 2. Load from local machine

## Step 2. Exploring the dataset

In [ ]:
df_train.head()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,2,100,100,100,2,100,100,61477,80,6,98136542,8,5,56,11601,20,0,7.0,5.656854,5840,0,2320.20,2436.833027,118.783480,0.132468,8.178045e+06,2.460000e+07,85700000,1,97900000,14000000.0,3.190000e+07,85700000,1,286190,71547.5,137766.35380,278139,181,0,0,172,112,0.081519,0.050949,0,5840,833.071429,1774.906302,3.150292e+06,0,0,0,0,1,0,0,0,0,897.153846,7.0,2320.20,8,56,5,11601,256,229,6,20,996.0,0.0,996,996,48900000.0,51900000.0,85700000,12200000,DDoS
1,2,23,13,23,2,13,33,49665,80,6,121917,3,4,687,361,681,0,229.0,391.454978,349,0,90.25,172.523187,8596.012041,57.416111,2.031950e+04,2.482528e+04,61854,1,91167,45583.5,2.300996e+04,61854,29313,92688,30896.0,52536.13923,91556,1,0,0,72,88,24.606905,32.809206,0,681,131.000000,253.090046,6.405457e+04,0,0,0,1,0,0,0,0,1,149.714286,229.0,90.25,3,687,4,361,8192,5061,2,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,1,2,3,58,1,59,7,59588,53,17,30773,2,2,74,262,37,37,37.0,0.000000,131,131,131.00,0.000000,10918.662460,129.984077,1.025767e+04,1.776074e+04,30766,3,3,3.0,0.000000e+00,3,3,4,4.0,0.00000,4,4,0,0,64,64,64.992038,64.992038,37,131,74.600000,51.485920,2.650800e+03,0,0,0,0,0,0,0,0,1,93.250000,37.0,131.00,2,74,2,262,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,2,1,2,1,5,5,2,80,50918,6,118,1,1,0,0,0,0,0.0,0.000000,0,0,0.00,0.000000,0.000000,16949.152540,1.180000e+02,0.000000e+00,118,118,0,0.0,0.000000e+00,0,0,0,0.0,0.00000,0,0,0,0,32,32,8474.576271,8474.576271,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,1,0.000000,0.0,0.00,1,0,1,0,905,229,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,2,4,2,8,2,2,4,55989,80,6,47,2,0,0,0,0,0,0.0,0.000000,0,0,0.00,0.000000,0.000000,42553.191490,4.700000e+01,0.000000e+00,47,47,47,47.0,0.000000e+00,47,47,0,0.0,0.00000,0,0,0,0,40,0,42553.191490,0.000000,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,0,0,0,0,0.000000,0.0,0.00,2,0,0,0,259,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


Count the number of attacks

In [ ]:
df_train['Label'].value_counts()

BENIGN                        490029
DoS Hulk                      161134
PortScan                      110992
DDoS                           89590
DoS GoldenEye                   7220
FTP-Patator                     5621
SSH-Patator                     4153
DoS slowloris                   4049
DoS Slowhttptest                3833
Bot                             1401
Web Attack � Brute Force        1059
Web Attack � XSS                 463
Infiltration                      25
Web Attack � Sql Injection        12
Heartbleed                         8
Name: Label, dtype: int64

In [ ]:
df_train.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,879589.000000,8.795890e+05,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,8.795890e+05,879589.000000,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05
mean,9.655316,37.867227,49.836973,48.006394,10.465226,55.566170,57.243626,42882.307181,6397.910982,8.693028,1.923037e+07,8.214027,8.823871,4.403727e+02,1.417082e+04,179.036862,13.701055,48.020170,61.747571,1463.559232,29.061780,487.495944,600.398140,1.154200e+06,7.835547e+04,1.785500e+06,4.593409e+06,1.505781e+07,1.817909e+05,1.895005e+07,3.497445e+06,5.784826e+06,1.493529e+07,8.764154e+05,9.546793e+06,1.832198e+06,2.096462e+06,5.847373e+06,7.775011e+05,0.035845,0.000069,-1.779193e+03,-1.391305e+03,7.066763e+04,7.731049e+03,11.697604,1520.444635,250.439207,478.990505,9.488540e+05,0.056930,0.035845,0.000164,0.348102,0.351718,0.068783,0.000069,0.000164,0.662657,277.045229,48.020170,487.495944,8.214027,4.403727e+02,8.823871,1.416974e+04,7118.105536,1463.521291,4.322587,-1.156058e+03,8.722149e+04,3.427464e+04,1.409675e+05,6.695474e+04,1.402000e+07,8.088585e+05,1.463478e+07,1.341685e+07
std,25.810487,40.954725,44.142062,39.181947,25.970811,41.115493,40.505493,19661.359246,16111.253716,4.734203,3.711919e+07,685.761027,912.882257,6.605092e+03,2.075330e+06,610.645248,57.039193,160.224075,237.180304,2657.258586,60.386127,814.918490,1167.665597,2.334998e+07,2.713128e+05,4.929222e+06,9.877418e+06,3.194961e+07,3.274780e+06,3.708712e+07,9.762596e+06,1.305067e+07,3.201589e+07,8.098459e+06,2.816566e+07,8.481475e+06,8.133173e+06,2.054948e+07,7.499528e+06,0.185904,0.008327,1.186150e+06,1.158914e+06,2.652601e+05,3.809361e+04,22.706568,2687.748532,394.185794,848.222439,2.302125e+06,0.231709,0.185904,0.012794,0.476369,0.477507,0.253085,0.008327,0.012794,0.643625,431.918903,160.224075,814.918490,685.761027,6.605092e+03,912.882257,2.075043e+06,13584.239160,7163.856980,

Read test and validation sets

In [ ]:
print('Test set: ')
df_test['Label'].value_counts()

Test set: 


BENIGN                        105019
DoS Hulk                       34547
PortScan                       23846
DDoS                           19271
DoS GoldenEye                   1542
FTP-Patator                     1178
DoS slowloris                    834
DoS Slowhttptest                 828
SSH-Patator                      826
Bot                              280
Web Attack � Brute Force         209
Web Attack � XSS                  93
Web Attack � Sql Injection         7
Heartbleed                         2
Infiltration                       1
Name: Label, dtype: int64

In [ ]:
df_test.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05
mean,9.625308,37.861701,49.864025,48.026984,10.434315,55.607434,57.249322,42825.772430,6398.028565,8.699867,1.937297e+07,6.393341,6.290955,425.507547,9.614426e+03,180.326422,13.616549,48.088141,62.258627,1471.358101,29.252240,489.581182,604.521501,1.141231e+06,7.836992e+04,1.791253e+06,4.628396e+06,1.515027e+07,1.736502e+05,1.910116e+07,3.521722e+06,5.821567e+06,1.502940e+07,8.824922e+05,9.685500e+06,1.863613e+06,2.133658e+06,5.939107e+06,7.890398e+05,0.036115,0.000095,-7.244541e+02,-7.290569e+02,7.076928e+04,7.712062e+03,11.748036,1528.642275,251.232699,481.683733,9.575668e+05,0.057278,0.036115,0.000164,0.348620,0.350392,0.068335,0.000095,0.000175,0.662680,277.964550,48.088141,489.581182,6.393341,425.507547,6.290955,9.615250e+03,7121.997130,1479.610888,3.475565,-4.185286e+02,8.921930e+04,3.533351e+04,1.448189e+05,6.831514e+04,1.412079e+07,8.001563e+05,1.473072e+07,1.352238e+07
std,25.759133,40.995800,44.175627,39.217990,25.916640,41.132627,40.537490,19694.807197,16123.799361,4.737968,3.724073e+07,364.106378,421.965728,3858.850242,1.454190e+06,618.937180,55.371692,158.738971,238.828812,2667.682650,62.105876,817.093491,1174.621985,2.348509e+07,2.727351e+05,4.872676e+06,9.915489e+06,3.202428e+07,3.155030e+06,3.721579e+07,9.783012e+06,1.309209e+07,3.209203e+07,8.108421e+06,2.836729e+07,8.531263e+06,8.212071e+06,2.070359e+07,7.526471e+06,0.186576,0.009772,3.865090e+05,3.865304e+05,2.665324e+05,3.733633e+04,23.000785,2699.393195,394.531874,851.827223,2.321479e+06,0.232374,0.186576,0.012824,0.476535,0.477094,0.252321,0.009772,0.013231,0.648263,432.349886,158.738971,817.093491,364.106378,3858.850242,421.965728,1.454292e+06,13565.260196,7247.906478,355.

In [ ]:
print('Validation set: ')
df_val['Label'].value_counts()

Validation set: 


BENIGN                        104952
DoS Hulk                       34443
PortScan                       23966
DDoS                           19164
DoS GoldenEye                   1531
FTP-Patator                     1136
SSH-Patator                      918
DoS slowloris                    913
DoS Slowhttptest                 838
Bot                              275
Web Attack � Brute Force         239
Web Attack � XSS                  96
Infiltration                      10
Web Attack � Sql Injection         2
Heartbleed                         1
Name: Label, dtype: int64

In [ ]:
df_val.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,188484.000000,1.884840e+05,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,1.884840e+05,188484.000000,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05
mean,9.731272,37.780172,49.883332,47.914115,10.539032,55.614689,57.218512,42898.015243,6422.896426,8.694239,1.924685e+07,7.434695,7.835328,4.390308e+02,1.180121e+04,178.156406,13.800163,48.024119,61.523877,1463.444457,29.060005,487.327309,600.629474,1.225907e+06,7.942923e+04,1.791008e+06,4.609148e+06,1.513562e+07,1.849447e+05,1.896832e+07,3.499186e+06,5.813441e+06,1.501155e+07,8.688593e+05,9.470943e+06,1.831885e+06,2.100258e+06,5.840577e+06,7.745564e+05,0.035764,0.000069,-6.292069e+04,-1.164520e+04,7.168020e+04,7.802082e+03,11.720783,1520.258568,249.863420,478.533085,9.479145e+05,0.056583,0.035764,0.000154,0.347573,0.352502,0.068372,0.000069,0.000154,0.660905,276.454458,48.024119,487.327309,7.434695,4.390308e+02,7.835328,1.179981e+04,7106.119968,1484.061920,3.641153,-8.963623e+03,9.042821e+04,3.565553e+04,1.450080e+05,6.899322e+04,1.407187e+07,8.205580e+05,1.469736e+07,1.346096e+07
std,25.915515,40.960165,44.147226,39.203217,26.082113,41.112314,40.531929,19646.086134,16145.734341,4.735494,3.709354e+07,561.529560,747.556849,6.996634e+03,1.668861e+06,604.186341,57.791361,160.261305,237.467628,2660.075880,60.839255,816.166175,1169.171476,2.482471e+07,2.735719e+05,4.871805e+06,9.865068e+06,3.198730e+07,3.220118e+06,3.705975e+07,9.655176e+06,1.307159e+07,3.205087e+07,7.956019e+06,2.804916e+07,8.424538e+06,8.163563e+06,2.053212e+07,7.404418e+06,0.185702,0.008305,2.253288e+07,3.503013e+06,2.672851e+05,3.867133e+04,22.334888,2689.274156,393.766139,847.928470,2.293114e+06,0.231045,0.185702,0.012403,0.476201,0.477751,0.252384,0.008305,0.012403,0.644010,431.481560,160.261305,816.166175,561.529560,6.996634e+03,747.556849,1.668515e+06,13575.776949,7232.426279,

## Step 3. Encode Datasets

Encoding the labels, and generate numpy array. Note that the label has not been encoded as one-hot coding. We will use one-hot code later. 

### Step 3.1 Encoding train dataset

In [ ]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
Xtrain = data.values
y_train = encode_label(df_label.values)

### Step 3.2. Encoding test dataset

In [ ]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

### Step 3.3 Encoding validation dataset

In [ ]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
Xval = data.values
y_val = encode_label(df_label.values)

## Step 4. Normalization or Standardization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. We implement the two methods to see the impact on the final classifications. 

## Option 1. Normalization

The values of the datasets are normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

### Step 4.1 Normalizing train dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(Xtrain)
X_train

array([[0.01010101, 1.        , 1.        , ..., 0.68650794, 0.71416667,
        0.10166667],
       [0.01010101, 0.22222222, 0.12121212, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01010101, 0.02020202, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.12121212, 0.12121212, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.87878788, 0.87878788, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ]])

### Step 4.2. Normalizing validation dataset

In [ ]:
X_val = scaler.transform(Xval)
X_val

array([[0.        , 0.32323232, 0.32323232, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05050505, 0.05050505, ..., 0.        , 0.        ,
        0.        ],
       [0.02020202, 0.09090909, 0.02020202, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.96969697, ..., 0.        , 0.        ,
        0.        ]])

### Step 4.3. Normalizing test dataset

In [ ]:
X_test = scaler.transform(Xtest)
X_test

array([[0.        , 0.96969697, 0.96969697, ..., 0.        , 0.71583333,
        0.71583333],
       [0.        , 0.96969697, 0.96969697, ..., 0.        , 0.6975    ,
        0.6975    ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.44444444, 0.08080808, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.78787879, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.28282828, 0.28282828, ..., 0.        , 0.        ,
        0.        ]])

## Option 2. Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(Xtrain)
X_val = scaler.transform(Xval)
X_test = scaler.transform(Xtest)

X_test

array([[-0.33534125,  1.44385799,  1.06843795, ..., -0.13390608,
         2.22765698,  2.34455089],
       [-0.33534125,  1.44385799,  1.06843795, ..., -0.13390608,
         2.15888788,  2.27338936],
       [-0.33534125, -0.90019522,  1.13640038, ..., -0.13390608,
        -0.45746381, -0.43398332],
       ...,
       [-0.33534125,  0.1741625 , -0.92512661, ..., -0.13390608,
        -0.45746381, -0.43398332],
       [-0.33534125, -0.90019522,  0.66066338, ..., -0.13390608,
        -0.45746381, -0.43398332],
       [-0.33534125, -0.21651303, -0.47204375, ..., -0.13390608,
        -0.45746381, -0.43398332]])

## Step 5 One-hot encoding for labels

y_train, y_test and y_val have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [ ]:
from tensorflow.keras.utils import to_categorical

Save the labels for AdaBoostClassifier

In [ ]:
y_train_ada = y_train
y_test_ada = y_test
y_val_ada = y_val

In [ ]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

## Step 6. Define the metrics

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

Instructions for updating:
non-resource variables are not supported in the long term


Get the attacks' names

In [ ]:
labels_d = make_value2index(df_test['Label'])

In [ ]:
print(labels_d)

{'BENIGN': 105018, 'Bot': 105298, 'DDoS': 124569, 'DoS GoldenEye': 126111, 'DoS Hulk': 160658, 'DoS Slowhttptest': 161486, 'DoS slowloris': 162320, 'FTP-Patator': 163498, 'Heartbleed': 163500, 'Infiltration': 163501, 'PortScan': 187347, 'SSH-Patator': 188173, 'Web Attack � Brute Force': 188382, 'Web Attack � Sql Injection': 188389, 'Web Attack � XSS': 188482}


# CNN1D Model 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout,MaxPooling1D
from tensorflow.keras.constraints import max_norm

In [ ]:
#hyper-params
batch_size = 7500 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 6
learning_rates = 1e-4
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

78
15


In [ ]:
X_train.shape

(879589, 78)

In [ ]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(879589, 78, 1)


In [ ]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(188483, 78, 1)


In [ ]:
X_val_r = np.zeros((len(X_val), input_dim, 1))
X_val_r[:, :, 0] = X_val[:, :input_dim]
print(X_val_r.shape)

(188484, 78, 1)


In [ ]:
type(X_train_r)

numpy.ndarray

In [ ]:
model = Sequential()

# input layer
model.add(Conv1D(filters=32, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(78,1)))
model.add(Conv1D(filters=32, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))

model.add(Conv1D(filters=64, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(Conv1D(filters=64, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1)) 

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 78, 32)            768       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 78, 32)            17440     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 39, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 39, 32)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 39, 32)            156       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 39, 64)            47168     
_________________________________________

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
]

metrics=METRICS

In [ ]:
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import tensorflow.keras
import time
time_start = time.time()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(moniter='val_loss',
                                              factor=0.1,
                                              patience=10)
nadam = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss = "categorical_crossentropy",optimizer = "nadam", metrics = metrics)

#model.compile(
#      optimizer=tf.keras.optimizers.Adam(lr=1e-4),
#      loss=tf.keras.losses.BinaryCrossentropy(),
#      metrics=metrics)
  

history = model.fit(X_train_r, y_train, 
                    epochs=6, 
                    batch_size=batch_size, 
                    verbose=2,
                    validation_data=(X_val_r, y_val),
                    callbacks=[reduce_lr])
time_end = time.time()
train_time = time_end - time_start
print("train_time:",train_time)

Train on 879589 samples, validate on 188484 samples
Epoch 1/6
879589/879589 - 8s - loss: 0.2499 - tp: 801396.0000 - fp: 33713.0000 - tn: 12280533.0000 - fn: 78193.0000 - accuracy: 0.9915 - precision: 0.9596 - recall: 0.9111 - val_loss: 0.0963 - val_tp: 183764.0000 - val_fp: 1578.0000 - val_tn: 2637198.0000 - val_fn: 4720.0000 - val_accuracy: 0.9978 - val_precision: 0.9915 - val_recall: 0.9750 - lr: 0.0010
Epoch 2/6
879589/879589 - 7s - loss: 0.0562 - tp: 862714.0000 - fp: 12465.0000 - tn: 12301781.0000 - fn: 16875.0000 - accuracy: 0.9978 - precision: 0.9858 - recall: 0.9808 - val_loss: 0.0372 - val_tp: 185975.0000 - val_fp: 2035.0000 - val_tn: 2636741.0000 - val_fn: 2509.0000 - val_accuracy: 0.9984 - val_precision: 0.9892 - val_recall: 0.9867 - lr: 0.0010
Epoch 3/6
879589/879589 - 7s - loss: 0.0430 - tp: 866445.0000 - fp: 10532.0000 - tn: 12303714.0000 - fn: 13144.0000 - accuracy: 0.9982 - precision: 0.9880 - recall: 0.9851 - val_loss: 0.0279 - val_tp: 186474.0000 - val_fp: 1903.0000 -

## Get the metrics

In [ ]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

In [ ]:
y_pred=model.predict(X_test_r)

In [ ]:
y_test_ada

array([ 4,  4, 10, ...,  0, 10,  0])

In [ ]:
display_metrics(y_test_ada, np.argmax(y_pred, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.68
Macro Recall: 0.70


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.69

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      0.99      0.99    105019
                       Bot       0.85      0.74      0.79       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       0.99      0.99      0.99      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      0.97      0.98       828
             DoS slowloris       0.96      0.99      0.98       834
               FTP-Patator       0.97      1.00      0.98      1178
                Heartbleed       0.00      0.00      0.00         2
              Infiltration       0.00      0.00      0.00         1
                  PortScan       0.96      0.99      0.98     23846
               SSH-Patator       0.93      0.98      0.96       826
  Web Attack � Brute Force       0.58      0.86      0.70       209
Web Attack � Sql Injection       0.00      0.00

features importance

In [ ]:
!pip install shap

     |████████████████████████████████| 358kB 3.8MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491618 sha256=eda5e3bc521ee6db3c77a9e7179d72c51515df58321e765e4424a6acc1a4aa84
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
import shap
import torch

In [ ]:
X_train_r

array([[[-0.29659729],
        [ 1.51710965],
        [ 1.13640038],
        ...,
        [ 8.4581104 ],
        [ 2.22140524],
        [-0.0393603 ]],

       [[-0.29659729],
        [-0.36301636],
        [-0.83451004],
        ...,
        [-0.13390608],
        [-0.45746381],
        [-0.43398332]],

       [[-0.33534125],
        [-0.87577799],
        [-1.06105146],
        ...,
        [-0.13390608],
        [-0.45746381],
        [-0.43398332]],

       ...,

       [[-0.33534125],
        [-0.60718856],
        [-0.83451004],
        ...,
        [-0.13390608],
        [-0.45746381],
        [-0.43398332]],

       [[-0.33534125],
        [ 1.224103  ],
        [ 0.86455067],
        ...,
        [-0.13390608],
        [-0.45746381],
        [-0.43398332]],

       [[ 3.50031109],
        [-0.90019522],
        [ 1.13640038],
        ...,
        [-0.13390608],
        [-0.45746381],
        [-0.43398332]]])

In [ ]:
e = shap.DeepExplainer(
        model, 
        X_train_r[np.random.choice(np.arange(len(X_train_r)), 5000, replace=False)]
        )

In [ ]:
x_samples = X_test_r[np.random.choice(np.arange(len(X_test_r)), 300, replace=False)]
print(len(X_test_r))
shap_values = e.shap_values(
    x_samples
)

188483


In [ ]:
shap_values

[array([[[ 1.88974128e-02],
         [ 2.44422039e-02],
         [ 7.90863260e-03],
         ...,
         [ 3.33903692e-04],
         [ 3.16640053e-03],
         [ 1.84838201e-04]],
 
        [[-7.07904705e-04],
         [-2.16842958e-02],
         [-3.10886542e-02],
         ...,
         [-5.71843478e-05],
         [-1.76225114e-02],
         [-2.95661783e-03]],
 
        [[ 1.36532482e-02],
         [ 2.30031684e-02],
         [ 6.37165371e-03],
         ...,
         [ 4.08133596e-04],
         [ 3.25749830e-03],
         [ 3.29690953e-04]],
 
        ...,
 
        [[ 1.96737497e-03],
         [ 1.72205100e-02],
         [ 1.76980348e-02],
         ...,
         [ 3.47539288e-05],
         [ 6.95759971e-03],
         [ 1.40195232e-03]],
 
        [[-2.28838858e-03],
         [ 9.50796786e-04],
         [ 3.53152688e-02],
         ...,
         [ 3.97209043e-05],
         [ 6.94573216e-03],
         [ 8.25892510e-04]],
 
        [[-2.18477619e-03],
         [-2.74482422e-02],
    

In [ ]:
len(shap_values) # classes

15

In [ ]:
len(shap_values[0]) # number of samples

300

In [ ]:
len(shap_values[0][0]) # number of features

78

In [ ]:
t= np.array(shap_values)

In [ ]:
t1=np.mean(np.abs(shap_values[0]), axis=0)

In [ ]:
t.shape

(15, 300, 78, 1)

In [ ]:
t1.shape

(78, 1)

In [ ]:
t1.T[0]

array([5.77372138e-03, 1.54758126e-02, 4.32355453e-02, 1.05614340e-02,
       9.74243982e-03, 5.16570459e-02, 2.06673207e-02, 2.23926765e-02,
       1.02967418e-02, 2.14363442e-02, 1.13646087e-02, 4.64861876e-05,
       1.24844442e-04, 7.14742011e-04, 3.50907086e-05, 5.04203100e-03,
       3.22929664e-03, 2.77239312e-03, 4.59308563e-03, 1.32713864e-02,
       1.34966493e-02, 7.60694979e-03, 1.10230949e-02, 3.68858739e-04,
       7.21612334e-03, 5.45970416e-03, 3.72214063e-03, 5.14767768e-03,
       3.76852431e-03, 3.95930057e-03, 3.65818860e-03, 4.02126235e-03,
       5.58797772e-03, 1.21997352e-03, 3.71841650e-03, 2.39059147e-03,
       8.88811294e-04, 2.59674371e-03, 8.65771791e-04, 2.21711930e-03,
       2.66035683e-05, 1.19498021e-06, 1.04020527e-06, 6.20478756e-03,
       7.72728956e-03, 1.83401994e-02, 6.70140946e-03, 8.72091969e-03,
       8.71832623e-03, 5.02025919e-03, 4.51317074e-03, 2.15088946e-03,
       3.55093225e-05, 2.39057114e-02, 2.07770673e-02, 5.89834260e-03,
      

In [ ]:
df = pd.DataFrame({
    "mean_abs_shap": np.mean(np.abs(shap_values[0]), axis=0).T[0],
    "stdev_abs_shap": np.std(np.abs(shap_values[0]), axis=0).T[0], 
    "features": col_names[:78]
})
df.sort_values("mean_abs_shap", ascending=False)[:10]

,mean_abs_shap,stdev_abs_shap,features
5,0.051657,0.035179,dst host count
2,0.043236,0.032913,dst src count
53,0.023906,0.021038,PSH Flag Count
7,0.022393,0.028723,Source Port
9,0.021436,0.018740,Protocol
54,0.020777,0.019942,ACK Flag Count
6,0.020667,0.023825,src host count
45,0.018340,0.015617,Min Packet Length
1,0.015476,0.010598,src dport count
66,0.015062,0.020032,Init_Win_bytes_forward


In [ ]:
df.values

array([[0.005773721375394485, 0.009589887132412581, 'dst sport count'],
       [0.015475812614733112, 0.010597735259723176, 'src dport count'],
       [0.04323554531969421, 0.03291346245016519, 'dst src count'],
       [0.010561433974160603, 0.010518842812620737, 'dport count'],
       [0.00974243982377772, 0.0175109605896722, 'sport count'],
       [0.051657045906696585, 0.0351786445098504, 'dst host count'],
       [0.020667320693132152, 0.02382521964605351, 'src host count'],
       [0.022392676473590896, 0.028722724374352127, 'Source Port'],
       [0.010296741846188017, 0.014717676409301123, 'Destination Port'],
       [0.021436344197834176, 0.01874043410101336, 'Protocol'],
       [0.011364608651373303, 0.011043851690852672, 'Flow Duration'],
       [4.64861876437057e-05, 0.00011764285134104959,
        'Total Fwd Packets'],
       [0.00012484444162508103, 0.001500690028397313,
        'Total Backward Packets'],
       [0.0007147420107327429, 0.0020658415582634035,
        'Total